In [2]:
%env KERAS_BACKEND = tensorflow

env: KERAS_BACKEND=tensorflow


In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# KERAS FUNCTION
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

#keras dataset
from keras.datasets import mnist

#keras utils function
from keras.utils import np_utils

/Users/James/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
#由keras 讀入minst
(x_train, y_train), (x_test, y_test)= mnist.load_data()

In [6]:
#資料的長相
print("總共有%d訓練資料，每筆資料的尺寸為%d x %d" %x_train.shape)
print("總共有%d測試資料，每筆資料的尺寸為%d x %d" %x_test.shape)

總共有60000訓練資料，每筆資料的尺寸為28 x 28
總共有10000測試資料，每筆資料的尺寸為28 x 28


In [7]:
#輸入格式整理
x_train = x_train.reshape(60000,28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

In [8]:
#將測試資料中0,1的抓出
x_train_01 = x_train[y_train <=1]
x_test_01 = x_test[y_test <=1]

In [9]:
y_train_10 = np_utils.to_categorical(y_train, 10)
y_test_10 = np_utils.to_categorical(y_test, 10)

y_train_01 = y_train[y_train<=1]
y_train_01 = np_utils.to_categorical(y_train_01, 2)

y_test_01 = y_test[y_test<=1]
y_test_01 = np_utils.to_categorical(y_test_01, 2)

In [10]:
#資料大小
x_train_01.shape, x_test_01.shape

((12665, 28, 28, 1), (2115, 28, 28, 1))

In [11]:
y_train_01.shape, y_test_01.shape

((12665, 2), (2115, 2))

In [12]:
#建立model
conv_layer = [Conv2D(32, (3, 3), padding='same', input_shape=(28,28,1)),Activation('relu'),MaxPooling2D(pool_size=(2,2)),
             Conv2D(64, (3, 3), padding='same', input_shape=(28,28,1)),Activation('relu'),MaxPooling2D(pool_size=(2,2)),
             Conv2D(128, (3, 3), padding='same', input_shape=(28,28,1)),Activation('relu'),MaxPooling2D(pool_size=(2,2))]
fc_layer = [Flatten(),Dense(200),Activation('relu'),Dense(10),Activation('softmax')]
model=Sequential(conv_layer+fc_layer)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 128)         73856     
__________

In [13]:
model.load_weights('handwriting_weights_cnn.h5')

In [14]:
new_fc_layer=[Flatten(),Dense(200),Activation('softmax'),Dense(2),Activation('softmax')]
model_0_to_1= Sequential(conv_layer+new_fc_layer)

In [15]:
for layer in conv_layer:
    layer.trainable = False

In [16]:
model_0_to_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 128)         73856     
__________

In [17]:
model_0_to_1.compile(loss='mse', optimizer=SGD(lr=0.1),metrics=['accuracy'])

In [18]:
model_0_to_1.fit(x_train_01, y_train_01, batch_size=100, epochs=7)

Epoch 1/7
12665/12665 [==============================] - 3s 247us/step - loss: 0.0983 - acc: 0.9885
Epoch 2/7
12665/12665 [==============================] - 3s 233us/step - loss: 0.0242 - acc: 0.9985
Epoch 3/7
12665/12665 [==============================] - 3s 233us/step - loss: 0.0126 - acc: 0.9992
Epoch 4/7
12665/12665 [==============================] - 3s 234us/step - loss: 0.0085 - acc: 0.9994
Epoch 5/7
12665/12665 [==============================] - 3s 234us/step - loss: 0.0063 - acc: 0.9995
Epoch 6/7
12665/12665 [==============================] - 3s 233us/step - loss: 0.0049 - acc: 0.9998
Epoch 7/7
12665/12665 [==============================] - 3s 233us/step - loss: 0.0041 - acc: 0.9998


In [19]:
score = model_0_to_1.evaluate(x_test_01, y_test_01)

2115/2115 [==============================] - 1s 266us/step


In [20]:
print('測試資料的loss:',score[0])
print('測試資料正確率:',score[1])

測試資料的loss: 0.003593532499652488
測試資料正確率: 1.0
